# Packages

In [3]:
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib as plt
import re 


# Import file and exploration

In [29]:
df = pd.read_csv("/Volumes/Extreme SSD/03_TheGraph/LP_UniswapV2_raw.csv")

In [6]:
df.head()
df.tail()
df.shape
df.info()
df.dtypes
df.count()
df.describe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65780 entries, 0 to 65779
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  65780 non-null  int64  
 1   poolId      65780 non-null  object 
 2   token0      65776 non-null  object 
 3   token1      65779 non-null  object 
 4   volumeUSD   65780 non-null  float64
 5   reserveUSD  65780 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 3.0+ MB


,Unnamed: 0,volumeUSD,reserveUSD
count,65780.000000,6.578000e+04,6.578000e+04
mean,498.195652,6.054236e+06,8.519737e+04
std,288.252205,2.478661e+08,7.247750e+06
min,0.000000,0.000000e+00,0.000000e+00
25%,249.000000,0.000000e+00,6.300000e-15
50%,498.000000,0.000000e+00,1.799500e-13
75%,747.000000,0.000000e+00,8.688717e+00
max,999.000000,3.835843e+10,1.743372e+09


In [7]:
# First Colum is repeating index values between 0-1000  from looping through the API. To be removed.
len(df.iloc[:,0].unique())

1000

In [30]:
# Remove first column
dfClean = df.drop("Unnamed: 0", axis =1)

In [31]:
# Checking columns for NULL values
print(df.isnull().sum())
print(dfClean.isnull().sum())

Unnamed: 0    0
poolId        0
token0        4
token1        1
volumeUSD     0
reserveUSD    0
dtype: int64
poolId        0
token0        4
token1        1
volumeUSD     0
reserveUSD    0
dtype: int64


# Removing Null values

In [10]:
# Null values for token0
null0 = df[df["token0"].isnull()]

In [11]:
# Null values for token1
null1 = df[df["token1"].isnull()]

In [12]:
# Cheeck position of example
dfClean.loc[40620]

poolId        0x9de60d24a3ac21d113d07e74845daf6259d0d536
token0                                               gm👶
token1                                              WETH
volumeUSD                                            0.0
reserveUSD                                           0.0
Name: 40620, dtype: object

In [12]:
# Create df with rows missing values
dfNull = null0.append(null1)

/var/folders/fw/z98v802x6fx6hbz_spqj2r740000gn/T/ipykernel_6210/1672124346.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfNull = null0.append(null1)


In [13]:
# Remove NULL observations
dfClean = dfClean.drop(dfNull.iloc[:,0].index)

In [14]:
# Check result table
print(dfClean.isnull().sum(), "\n")

print("Shape: ", dfClean.shape)

poolId        0
token0        0
token1        0
volumeUSD     0
reserveUSD    0
dtype: int64 

Shape:  (65775, 5)


# Cleaning table

In [15]:
# Remove commas from value strings in columns that are object-type
for i in range(len(dfClean.columns)):
    if (dfClean.dtypes[i]) == "object":
        dfClean.iloc[:,i] = dfClean.iloc[:,i].replace(",", "", regex=True)
        dfClean.iloc[:,i] = dfClean.iloc[:,i].replace('"', "", regex=True)
        dfClean.iloc[:,i] = dfClean.iloc[:,i].replace(r'"', "", regex=True)
        dfClean.iloc[:,i] = dfClean.iloc[:,i].replace(r'<.+>', "", regex=True)
        dfClean.iloc[:,i] = dfClean.iloc[:,i].replace("<", "", regex=True)
        dfClean.iloc[:,i] = dfClean.iloc[:,i].replace(">", "", regex=True)


In [32]:
# Add flag column
dfClean["UniswapV2_LP"] = 1

# Add UniswapV2Router Address

In [33]:
#dfClean.loc[len(dfClean.index)] = ['test', 1] 
dfTemp = {'poolId': '0x7a250d5630b4cf539739df2c5dacb4c659f2488d', 'UniswapV2_LP': 1}
dfClean = dfClean.append(dfTemp, ignore_index = True)

/var/folders/fw/z98v802x6fx6hbz_spqj2r740000gn/T/ipykernel_2118/2790402635.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfClean = dfClean.append(dfTemp, ignore_index = True)


# Export as csv



In [ ]:
# Export all columns from cleaned df. Doesnt work for upload to BQ. Use '_export'
dfClean.to_csv("/Users/marcvendramet/Desktop/HSG/Master/MA/08_Data/04_TheGraph/LP_UniswapV2_clean.csv"

In [34]:
# Export only two columns for useabulity on BigQuery
dfExport = dfClean[["poolId", "UniswapV2_LP"]]

In [39]:
dfExport.to_csv("/Volumes/Extreme SSD/03_TheGraph/LP_UniswapV2_export.csv", index_label = "index", index = True)

## Archived: Labels not needed
clabels = dfClean.columns.tolist()
clabels.insert(0, "index")
print(clabels)
